<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/03-ss3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JAIIO: BoW+SS3

En esta notebook se presetan los experimentos sobre la estrategia de representación y técnica de aprendizaje *baseline* utilizada para las 50 JAIIO.

Para ello vamos a preprocesar los correos y aplicar:
- Bag of words,
- SS3 como indicador de las _features_ importantes.


## Instalación y Carga de librerías y funciones útiles

### Instalación de librerías

Se instalan las librerías que no están en el entorno de Google Colab

In [1]:
# Se instala gensim que es el que tiene el modelo Word2Vec
!pip install requests
!pip install wget

### Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [2]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset, consolidar_df

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [3]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos

### Carga de datos

Se carga el dataframe en memoria con el preprocesamiento de los datos:

In [4]:
import warnings
from os import path
warnings.filterwarnings("ignore")

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/'
TRAIN_FILE = 'correos-train-80.csv'
TEST_FILE = 'correos-test-20.csv'

# Chequeo sobre si los archivos están en el working directory
download_files = not(path.exists(TRAIN_FILE))

etiquetas = get_clases()

# Defino la cantidad de clases a utilizar
CANTIDAD_CLASES = len(etiquetas)

train_df, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

# Se ejecuta el preprocesamiento de correos sobre el campo Consulta de train y test
import pandas as pd
train_df['Consulta'] = pd.Series(preprocesar_correos(train_df['Consulta']))
test_df['Consulta'] = pd.Series(preprocesar_correos(test_df['Consulta']))

# Cambio los integers por las etiquetas
train_df['clase'] = etiquetas[train_df['clase']]
test_df['clase'] = etiquetas[test_df['clase']]

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(train_df.clase.unique()), train_df.clase.unique()))


El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 20 clases: ['Inscripción a Cursadas' 'Cambio de Carrera' 'Reincorporación'
 'Ingreso a la Universidad' 'Boleto Universitario'
 'Pedido de Certificados' 'Exámenes' 'Requisitos de Ingreso' 'Cursadas'
 'Situación Académica' 'Vacunas Enfermería' 'Consulta por Legajo'
 'Problemas con la Clave' 'Consulta sobre Título Universitario'
 'Certificados Web' 'Carga de Notas' 'Otras Consultas'
 'Cambio de Comisión' 'Consulta por Equivalencias' 'Datos Personales'].


## SS3

Se instala la librería:

In [5]:
pip install pyss3

Se instancia la técnica de __SS3__ y se ajustan los hiperparámetros:

In [6]:
from sklearn.preprocessing import MinMaxScaler
from pyss3 import SS3
from pyss3.util import Evaluation, span
from pyss3.server import Live_Test

clf = SS3()

clf.train(train_df['Consulta'], train_df['clase'], prep=False)

best_s, best_l, best_p, best_a = Evaluation.grid_search(
  clf, test_df['Consulta'], test_df['clase'],
  s=span(0.2, 0.8, 6),
  l=span(0.1, 2, 6),
  p=span(0.5, 2, 6),
  a=[0, .1, .2],
  prep=False,  # <- do not forget to disable default preprocessing
  tag="grid search (test)"
)

print("Smoothness(s):", best_s)
print("Significance(l):", best_l)
print("Sanction(p):", best_p)
print("Alpha(a):", best_a)

Evaluation.plot()

Grid search: 100%|██████████| 648/648 [00:00<00:00, 84291.93it/s]


Smoothness(s): 0.44
Significance(l): 0.1
Sanction(p): 0.5
Alpha(a): 0.0


True

Se genera el modelo en base a los mejores hiperparámetros encontrados:

In [7]:
from sklearn.metrics import accuracy_score

clf.set_hyperparameters(0.44, 0.1, 0.5, 0.0)
y_pred = clf.predict(test_df['Consulta'], prep=False)

accuracy = accuracy_score(y_pred, test_df['clase'])
print("\nAccuracy was:", accuracy)

Classification: 100%|██████████| 200/200 [00:00<00:00, 31905.55it/s]


Accuracy was: 0.715


Se verifica el __Test Live__:

In [ ]:
Live_Test.run(clf, test_df['Consulta'], test_df['clase'])

# Referencias
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05